In [ ]:
# @title Download Checkpoint File and Install Python Packages
!gdown 1F3jUW2WQyIGbiPqUTcyE4zpd38v0sY7V

!pip install --no-cache-dir --upgrade \
  'git+https://github.com/librosa/librosa.git' \
  'git+https://github.com/bmcfee/resampy.git' \
  'git+https://github.com/tastyForReal/piano_transcription_inference.git' \
  'git+https://github.com/ytdl-org/youtube-dl.git' \
  'git+https://github.com/spotDL/spotify-downloader.git' \
  > '/dev/null' 2>&1

In [ ]:
# @title Create Form
import os
import shutil
import subprocess
import ipywidgets as widgets
from google.colab import files as colab_files
from datetime import datetime
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio

for path, subdirs, files in os.walk('.', topdown=False):
    for subdir in subdirs:
        directory = os.path.join(path, subdir)
        if not os.listdir(directory):
            os.rmdir(directory)
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension in (".mp3", ".mid", ".7z"):
            os.remove(os.path.join(path, file))

download_method = widgets.ToggleButtons(
    options=['spotdl', 'youtube-dl'],
    description='Method:',
    disabled=False,
    button_style='',
)

input_text = widgets.Text(value='https://www.youtube.com/watch?v=VgX9bb425Cc', description='Input:', disabled=False)

convert_button = widgets.Button(
    description='Convert',
    disabled=False,
    button_style='',
)

display(download_method, input_text, convert_button)


def convert_music(btn):

    download_method_choice = download_method.value
    input_url = input_text.value

    convert_button.disabled = True
    print('Starting download process...')

    if download_method_choice == 'spotdl':
        subprocess.run(['spotdl', input_url])
    elif download_method_choice == 'youtube-dl':
        subprocess.run([
            'youtube-dl', '-x', '--audio-format', 'mp3', '-f',
            '(bestvideo+bestaudio/best)[protocol^=http][ext=mp4]', input_url
        ])
    else:
        print("Invalid download method selected.")

    mp3_files = [f for f in os.listdir() if f.endswith('.mp3')]

    if not mp3_files:
        print("No MP3 files found. Stopping script.")
        return

    midi_folder = datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')
    os.mkdir(midi_folder)

    print('Download complete. Starting conversion process...')

    for mp3_file in mp3_files:
        midi_file = datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')
        midi_file_path = f'/content/{midi_folder}/{mp3_file}_{midi_file}.mid'

        (audio, _) = load_audio(mp3_file, sr=sample_rate, mono=True)

        transcriptor = PianoTranscription(
            device='cuda',
            checkpoint_path='/content/note_F1=0.9677_pedal_F1=0.9186.pth')
        transcribed_dict = transcriptor.transcribe(audio, midi_file_path)

    if shutil.which('7z') is None:
        print("7z executable not found. Stopping script.")
        return
    archive_name = f"{midi_folder}.7z"
    subprocess.run(["7z", "a", "-t7z", "-mx=9", archive_name, midi_folder])

    colab_files.download(archive_name)

    convert_button.disabled = False

    print(f"MIDI files created as {archive_name}")


convert_button.on_click(convert_music)